# **Purpose**
This notebook is a tutorial for ZapMeNot.

# **Configure for the Tutorial**
This step won't be required for production work.  The module search path is being altered to preferentially run
the "nightly builds" of both WatpropPy and ZapMeNot.

In [3]:
import sys
sys.path.insert(0,'/home/alan025/bin/WatpropPy')
sys.path.insert(0,'/home/alan025/bin/ZapMeNot')


# **Import packages**
The Python packages required by this notebook are imported. The package and Python interpreter versions are printed to ensure analysis reproducibility. 

In [5]:
from IPython.display import display, Markdown, Math, Latex, Javascript, HTML
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import math
import zap_me_not
# import WatpropPy

# **Overpower Trip High Range Uncertainty**
Calculation EE-0063 Rev. 2 defines the Channel Statistical Accuracy (CSA) as

${\scriptstyle CSA = SE \pm \sqrt{EA^{2} + PMA_{1}^2 + PMA_{2}^2 + PMA_{3}^2 + (M1 + M1MTE)^2 + (M5 + M5MTE)^2 + RD^2 + RTE^2 + RRA^2}}$

The same reference defines each term in units of % of span

* SE = 0.000% of span
* EA = 0.000% of span
* PMA<sub>1</sub> = 1.667% of span
* PMA<sub>2</sub> = 4.167% of span
* PMA<sub>3</sub> = 1.667% of span
* M1 = 0.100% of span
* M1MTE = 0.110% of span
* M5 = 0.833% of span
* M5MTE = 0.943% of span
* RD = 1.000% of span
* RTE = 0.5% of span
* RRA = 0.417% of span

As shown in EE-0063 Rev. 2, values can be converted to %RTP by multiplying by the factor 120%/100%.

In [ ]:
factor = 120./100.
SE = 0 * factor
EA = 0 * factor
PMA1 = 1.667 * factor
PMA2 = 4.167 * factor
PMA3 = 1.667 * factor
M1 = 0.1 * factor
M1MTE = 0.11 * factor
M5 = 0.833 * factor
M5MTE = 0.943 * factor
RD = 1 * factor
RTE = 0.5 * factor
RRA = 0.417 * factor
display(Markdown(f"**SE = {SE:2.3} %RTP**"))
display(Markdown(f"**EA = {EA:2.3} %RTP**"))
display(Markdown(f"**PMA1 = {PMA1:2.3} %RTP**"))
display(Markdown(f"**PMA2 = {PMA2:2.3} %RTP**"))
display(Markdown(f"**PMA3 = {PMA3:2.3} %RTP**"))
display(Markdown(f"**M1 = {M1:2.3} %RTP**"))
display(Markdown(f"**M1MTE = {M1MTE:2.3} %RTP**"))
display(Markdown(f"**M5 = {M5:2.3} %RTP**"))
display(Markdown(f"**M5MTE = {M5MTE:2.3} %RTP**"))
display(Markdown(f"**RD = {RD:2.3} %RTP**"))
display(Markdown(f"**RTE = {RTE:2.3} %RTP**"))
display(Markdown(f"**RRA = {RRA:2.3} %RTP**"))


# **Calorimetric Uncertainty**
A revised calorimetric uncertatinty as a function of power, PMA<sub>1</sub>, is defined in Calculation SM-1128, Rev. 1, Addendum B.  The following work fits a cubic spline to the data and plots the results to ensure the spline reasonably represents the original data.  The spline function PMA1( ) will be used in subsequent calculations.

In [ ]:
calorimetric_power_levels = [101.7, 100, 95, 90, 85, 80, 75, 70, 65, 
                             60, 55, 50, 45, 40, 35, 30, 25, 20]
calorimetric_uncertainties = [0.782, 0.791, 0.822, 0.862, 0.911, 0.971, 1.044, 1.132, 
                              1.236, 1.362, 1.514, 1.7, 1.929, 2.219, 2.596, 3.102, 
                              3.822, 4.941]
# create the interpolation function PMA1( )
PMA1 = interpolate.interp1d(calorimetric_power_levels, 
                            calorimetric_uncertainties,kind='cubic') 
# define an array of power levels to use as interpolation inpu
xnew = np.arange(20, 101, 1) 
# interpolate a calorimetric uncertainty at each power level in the array xnew
ynew = PMA1(xnew) 
plt.figure(figsize=(11,7))
plt.plot(calorimetric_power_levels, calorimetric_uncertainties, 'o', xnew, ynew, '-')
plt.xlabel('Calorimetric Power Level (%RTP)')
plt.ylabel('Calorimetric Uncert. (%RTP)')
plt.show()

# **Assumptions**
Additional uncertainties used in SM-1875 Rev 0 are:

* NIS downcomer temperature coefficient is negligible and $\alpha\Delta T = 0$
* Radial power distribution effect ($f(P)$) = ±4% indicated power


# **Calculations**
From SM-1875 Appendix A, the CSA that includes a reduced-power normalization is

$CSA=\alpha\Delta T\left(1-P\right) \pm \sqrt{\left(\frac{PMA_1}{P}\right)^2+\left(PMA_2\right)^2+\left(\frac{PMA_3}{P}\right)^2+f\left(P\right)^2+RE^2}$


where P is the *fractional* reduced power level at which a calorimetric and subsequent power range channel calibration were performed.


RE represents a combination of other calibration terms:

$RE = \sqrt{(M1 + M1MTE)^2 + (M5 + M5MTE)^2 + RD^2 + RTE^2 + RRA^2 }$


In [ ]:
RE = math.sqrt((M1+M1MTE)**2 + (M5+M5MTE)**2 + RD**2 + RTE**2 + RRA**2)
display(Markdown(f"**RE = {RE:2.5} %RTP**"))


Applying the assumptions listed previously, the CSA equation following a reduced-power normalization is:

$CSA= \sqrt{\left(\frac{PMA_1}{P}\right)^2+\left(PMA_2\right)^2+\left(\frac{PMA_3}{P}\right)^2+f\left(P\right)^2+RE^2}$




In [ ]:
# define an array of power levels to use as interpolation input
Fp = 4.0  # this is the radial power distribution effect, assumed to be a constant
power = np.arange(20, 101, 1) 
fracPower = power/100
CSA = np.sqrt((PMA1(power)/fracPower)**2 + PMA2**2 + (PMA3/fracPower)**2 + Fp**2 + RE**2)
plt.figure(figsize=(12,8))
plt.plot(power, CSA)
plt.xlabel('Calibration Power Level (%RTP)')
plt.ylabel('CSA (%RTP)')
plt.grid()
plt.show()

The required trip setpoint, TS, in %RTP is defined in SM-1875 as:

TS ≤ minimum of (109% RTP, (118% RTP – CSA)),

or

In [ ]:
TS = np.minimum(109, 118-CSA)
plt.figure(figsize=(12,8))
plt.plot(power, TS)
plt.xlabel('Calibration Power Level (%RTP)')
plt.ylabel('Trip Setpoint (%RTP')
plt.xlim(20, 50)
plt.yticks([90,92.5,95,97.5,100,102.5,105,107.5,109])
plt.xticks([20,25,30,35,40,45,50,47])
plt.grid()
plt.show()